In [1]:
import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
sys.path.append('../../src/')
from train import load_data
# from diffusion import DiffusionModel
# from evaluate import get_results
from omegaconf import OmegaConf
# from main import load_data, make_model
import numpy as np
import os
import glob
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
import torch
from model2 import Autoencoder, Preprocessor, Discriminator
import magic
import torch
import pathlib
import copy

import wandb
import sys
import matplotlib.pyplot as plt
import scprep
import pandas as pd
from omegaconf import OmegaConf
import os
import glob
import numpy as np
import torch
from torch import nn
import torch.optim as optim
from geodesic import jacobian, velocity, CondCurve, GeodesicBridgeOverfit
from plotly3d.plot import scatter, trajectories
import torch
from torch import nn
import pytorch_lightning as pl
from procrustes import Procrustes
from torch.utils.data import DataLoader, TensorDataset

import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

wandb.login()
api = wandb.Api()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xingzhis. Use `wandb login --relogin` to force relogin


In [2]:
def _distance_to_geodesic_criterion(predicted_geodesic, true_geodesic):
    # the inputs here are single samples from a geodesic; should be shape num_samples x num_dims
    # for each input point, we want the closest distance to any point on the true geodesic using the euclidean distance, torch.cdist
    D = torch.cdist(predicted_geodesic, true_geodesic)
    min_dists_to_true_geodesic = D.min(dim=1)[0]
    # we take the mean of the squared distances
    return torch.mean(min_dists_to_true_geodesic**2)
# def distance_to_geodesic_criterion(
#     predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
#     true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
#     ):
#     """
#     Mean of the squared distances from each predicted point to the closest point on the true geodesic
#     """
#     dists = []
#     for i in range(predicted_geodesic.shape[0]):
#         dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
#     dists = torch.stack(dists)
#     return dists.mean()

def distance_to_geodesic_criterion_len(
    predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
    true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
    lengths=1.,
    ):
    """
    Mean of the squared distances from each predicted point to the closest point on the true geodesic
    """
    dists = []
    # for i in range(predicted_geodesic.shape[0]):
    for i in range(len(predicted_geodesic)):
        dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
    dists = torch.stack(dists)
    dists = dists / lengths
    return dists.mean()

def distances_to_geodesic(
    predicted_geodesic:torch.Tensor, # size num_geodesics x num_samples x num_dims
    true_geodesic:torch.Tensor, # size num_geodesics num_samples x num_dims. But it's okay if the num_samples are different
    ):
    """
    Mean of the squared distances from each predicted point to the closest point on the true geodesic
    """
    dists = []
    # for i in range(predicted_geodesic.shape[0]):
    for i in range(len(predicted_geodesic)):
        dists.append(_distance_to_geodesic_criterion(predicted_geodesic[i], true_geodesic[i]))
    dists = torch.stack(dists)
    return dists

In [3]:
entity = "xingzhis"
project = "dmae"
# sweep_id = 'a72qr26q'
sweep_id = 'dcxgbhjp'
sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
runs_data = []

# Iterate through each run in the sweep
for run in sweep.runs:
    # Extract metrics and configs
    metrics = run.summary._json_dict
    configs = run.config
    
    # Combine metrics and configs, and add run ID
    combined_data = {**metrics, **configs, "run_id": run.id}
    
    # Append the combined data to the list
    runs_data.append(combined_data)

# Create a DataFrame from the runs data
df = pd.DataFrame(runs_data)

In [4]:
names = df[(df['loss_epoch']!='NaN')][['data_name']].values.flatten()

In [5]:
df[(df['loss_epoch']=='NaN')][['data_name']].values.flatten()

array([], dtype=object)

In [13]:

res_list = []
missing = []
failed = []

for data_name in tqdm(names):
    try:
        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'ys48kno0'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        # run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
        run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == 1.0) & (df['dimensions.latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('AE not found:', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        # run = api.run(f"{entity}/{project}/{run_id}")
        folder_path = '../../src/wandb/'
        cfg = OmegaConf.create(run.config)
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]
        cfg.data.root = '../' + cfg.data.root
        model = Autoencoder.load_from_checkpoint(ckpt_path)
        data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)



        # sweep_id = 'ywep3ixr'
        # sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        # # Initialize an empty list to store run data
        # runs_data = []

        # # Iterate through each run in the sweep
        # for run in sweep.runs:
        #     # Extract metrics and configs
        #     metrics = run.summary._json_dict
        #     configs = run.config
            
        #     # Combine metrics and configs, and add run ID
        #     combined_data = {**metrics, **configs, "run_id": run.id}
            
        #     # Append the combined data to the list
        #     runs_data.append(combined_data)

        # # Create a DataFrame from the runs data
        # df = pd.DataFrame(runs_data)
        # run_ids = df[(df['data.name'] == data_name)][['run_id']]
        # assert len(run_ids) == 1
        # run_id = run_ids.iloc[0]
        # run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
        # folder_path = '../../src/wandb/'
        # cfg = OmegaConf.create(run.config)
        # folder_list = glob.glob(f"{folder_path}*{run.id}*")
        # ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        # ckpt_path = ckpt_files[0]
        # cfg.data.root = '../' + cfg.data.root
        # discriminator = Discriminator.load_from_checkpoint(ckpt_path)



        entity = "xingzhis"
        project = "dmae"
        # sweep_id = 'a72qr26q'
        sweep_id = 'o5lf5v7t'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        run_ids = df[(df['data_name'] == data_name) & (df['loss_epoch'] != 'NaN') & (df['dimensions_latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) < 1:
            print('Geod not found', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        cfg_main = OmegaConf.create(run.config)
        folder_path = '../geodesic_on_datasets//wandb/'
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]

        x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
        xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        # if cfg_main.overfit:
        #     ids = torch.eye(xbatch.size(0))
        # else:
        ids = torch.zeros((xbatch.size(0),1))
        dataset = TensorDataset(xbatch, xendbatch, ids)
        dataloader = DataLoader(dataset, batch_size=len(x), shuffle=False)

        for param in model.encoder.parameters():
            param.requires_grad = False
        # def func(x):
        #     return model.encoder(x)
        # for param in discriminator.parameters():
        #     param.requires_grad = False
        # def discriminator_func_for_grad(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        # def discriminator_func(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        ofm = lambda x: x

        checkpoint = torch.load(ckpt_path)

        # Remove the "data_pts" key from the state_dict
        if "data_pts" in checkpoint['state_dict']:
            del checkpoint['state_dict']['data_pts']

        gbmodel = GeodesicBridgeOverfit(
            func=ofm,
            # func = enc_func,
            # discriminator_func=disc_func_pen,
            # discriminator_func_for_grad=discriminator_func_for_grad,
            input_dim=x.size(1), 
            hidden_dim=64, 
            scale_factor=1, 
            symmetric=True, 
            num_layers=3, 
            n_tsteps=100, 
            lr=1e-3, 
            weight_decay=1e-3,
            discriminator_weight=0.,
            discriminator_func_for_grad_weight=0.,
            id_dim=1,
            id_emb_dim=1,
            density_weight=0.,
            length_weight=1.,
        )

        gbmodel.load_state_dict(checkpoint['state_dict'])

        # batch = next(iter(dataloader))
        # x0, x1, ids = batch
        try:
            data_gt = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/data/neurips_results/toy/gt/{data_name}.npz", allow_pickle=True)
        except:
            print(f"CANNOT FIND FILE {data_name}.npz")
            missing.append(data_name)
            continue
        xbatch = torch.tensor(data_gt['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data_gt['end_points'], dtype=x.dtype, device=x.device)
        x0 = xbatch
        x1 = xendbatch
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        ids = torch.zeros((xbatch.size(0),1))
        # ids = torch.eye((xbatch.size(0)))

        # dataset = TensorDataset(xbatch, xendbatch, ids)
        # dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

        def cc_func(x0, x1, t):
            return gbmodel.cc(x0, x1, t, ids)
        vectors = velocity(cc_func, gbmodel.ts, x0, x1)
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
        vectors_flat = vectors.flatten(0,1)
        cc_pts_flat = cc_pts.flatten(0, 1)
        jac_flat = jacobian(gbmodel.func, cc_pts_flat)
        length_all = torch.sqrt((torch.einsum("nij,nj->ni", jac_flat, vectors_flat)**2).sum(axis=1))
        length_all = length_all.reshape(vectors.shape[0], vectors.shape[1])
        length = length_all.mean(axis=0)

        geods = (cc_pts_flat).reshape(cc_pts.shape)
        length2 = torch.sqrt(torch.diff(geods, axis=0)**2).sum(axis=-1).sum(axis=0)


        # plt.scatter(length2.detach().numpy(), data_gt['geodesic_lengths'])
        # plt.title(data_name)
        # plt.show()

        # gt_len = torch.tensor(data_gt['geodesic_lengths'])
        true_geod = torch.tensor(data_gt['geodesics'], dtype=torch.float32)
        gt_len = torch.sqrt(torch.diff(true_geod.permute(1,0,2), axis=0)**2).sum(axis=-1).sum(axis=0)
        corr = np.corrcoef(length.detach().numpy(), gt_len.cpu().numpy())[0,1]
        mse = ((length.detach().numpy() - gt_len.cpu().numpy())**2).mean()
        dist2geod = distance_to_geodesic_criterion_len(geods.permute(1,0,2), true_geod, lengths=gt_len).detach().numpy()

        res_list.append(dict(
            name=data_name,
            length_corr=corr,
            length_mse=mse,
            dist2geod=dist2geod
        ))
    except Exception as e:
        failed.append(data_name)
        print(e)

res_df = pd.DataFrame(res_list)


  0%|          | 0/36 [00:00<?, ?it/s]/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
  6%|▌         | 2/36 [00:00<00:06,  5.45it/s]

Geod not found Series([], Name: run_id, dtype: object) torus_15_0.7
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) torus_15_0.1
single positional indexer is out-of-bounds


 14%|█▍        | 5/36 [00:00<00:04,  7.73it/s]

Geod not found Series([], Name: run_id, dtype: object) torus_50_0.1
single positional indexer is out-of-bounds


 17%|█▋        | 6/36 [00:01<00:06,  4.46it/s]

...

 58%|█████▊    | 21/36 [00:06<00:05,  2.72it/s]

Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.3
single positional indexer is out-of-bounds


 67%|██████▋   | 24/36 [00:07<00:04,  2.68it/s]

Geod not found Series([], Name: run_id, dtype: object) ellipsoid_15_0.5
single positional indexer is out-of-bounds


100%|██████████| 36/36 [00:13<00:00,  2.64it/s]


In [14]:
failed

['torus_15_0.7',
 'torus_15_0.1',
 'torus_50_0.1',
 'torus_15_0.5',
 'saddle_15_0.1',
 'saddle_15_0',
 'saddle_10_0.1',
 'saddle_50_0.1',
 'hemisphere_15_0.3',
 'ellipsoid_15_0.5']

In [8]:

res_list = []
missing = []
failed2 = []

for data_name in tqdm(failed):
    try:
        entity = "xingzhis"
        project = "dmae"
        sweep_id = 'ys48kno0'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")

        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        # run_ids = df[(df['data.name'] == data_name) & (df['cfg/loss/weights/cycle'] == 1.) & (df['cfg/dimensions/latent'] == 3)]['run_id']
        run_ids = df[(df['data.name'] == data_name) & (df['loss.weights.cycle'] == 1.0) & (df['dimensions.latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('AE not found:', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        # run = api.run(f"{entity}/{project}/{run_id}")
        folder_path = '../../src/wandb/'
        cfg = OmegaConf.create(run.config)
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]
        cfg.data.root = '../' + cfg.data.root
        model = Autoencoder.load_from_checkpoint(ckpt_path)
        data = np.load(f"{cfg.data.root}/{cfg.data.name}{cfg.data.filetype}", allow_pickle=True)



        # sweep_id = 'ywep3ixr'
        # sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        # # Initialize an empty list to store run data
        # runs_data = []

        # # Iterate through each run in the sweep
        # for run in sweep.runs:
        #     # Extract metrics and configs
        #     metrics = run.summary._json_dict
        #     configs = run.config
            
        #     # Combine metrics and configs, and add run ID
        #     combined_data = {**metrics, **configs, "run_id": run.id}
            
        #     # Append the combined data to the list
        #     runs_data.append(combined_data)

        # # Create a DataFrame from the runs data
        # df = pd.DataFrame(runs_data)
        # run_ids = df[(df['data.name'] == data_name)][['run_id']]
        # assert len(run_ids) == 1
        # run_id = run_ids.iloc[0]
        # run = api.run(f"{entity}/{project}/{run_ids.iloc[0].values[0]}")
        # folder_path = '../../src/wandb/'
        # cfg = OmegaConf.create(run.config)
        # folder_list = glob.glob(f"{folder_path}*{run.id}*")
        # ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        # ckpt_path = ckpt_files[0]
        # cfg.data.root = '../' + cfg.data.root
        # discriminator = Discriminator.load_from_checkpoint(ckpt_path)



        entity = "xingzhis"
        project = "dmae"
        # sweep_id = '88x8glfh'
        sweep_id = 'jenfsv5r'
        sweep = api.sweep(f"{entity}/{project}/{sweep_id}")
        runs_data = []

        # Iterate through each run in the sweep
        for run in sweep.runs:
            # Extract metrics and configs
            metrics = run.summary._json_dict
            configs = run.config
            
            # Combine metrics and configs, and add run ID
            combined_data = {**metrics, **configs, "run_id": run.id}
            
            # Append the combined data to the list
            runs_data.append(combined_data)

        # Create a DataFrame from the runs data
        df = pd.DataFrame(runs_data)

        run_ids = df[(df['data_name'] == data_name) & (df['loss_epoch'] != 'NaN') & (df['dimensions_latent'] == 3)]['run_id']
        # assert len(run_ids) == 1
        if len(run_ids) != 1:
            print('Geod not found', run_ids, data_name)
        run_id = run_ids.iloc[0]
        run = api.run(f"{entity}/{project}/{run_id}")
        cfg_main = OmegaConf.create(run.config)
        folder_path = '../geodesic_on_datasets//wandb/'
        folder_list = glob.glob(f"{folder_path}*{run.id}*")
        ckpt_files = glob.glob(f"{folder_list[0]}/files/*.ckpt")
        ckpt_path = ckpt_files[0]

        x = torch.tensor(data['data'], dtype=torch.float32, device=model.device)
        xbatch = torch.tensor(data['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data['end_points'], dtype=x.dtype, device=x.device)
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        # if cfg_main.overfit:
        #     ids = torch.eye(xbatch.size(0))
        # else:
        ids = torch.zeros((xbatch.size(0),1))
        dataset = TensorDataset(xbatch, xendbatch, ids)
        dataloader = DataLoader(dataset, batch_size=len(x), shuffle=False)

        for param in model.encoder.parameters():
            param.requires_grad = False
        # def func(x):
        #     return model.encoder(x)
        # for param in discriminator.parameters():
        #     param.requires_grad = False
        # def discriminator_func_for_grad(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        # def discriminator_func(x):
        #     return discriminator.positive_proba(x, normalize=False).reshape(-1,1)
        ofm = lambda x: x
        gbmodel = GeodesicBridgeOverfit.load_from_checkpoint(
            checkpoint_path=ckpt_path,
            func=ofm,
            # func = enc_func,
            # discriminator_func=disc_func_pen,
            # discriminator_func_for_grad=discriminator_func_for_grad,
            input_dim=x.size(1), 
            hidden_dim=64, 
            scale_factor=1, 
            symmetric=True, 
            num_layers=3, 
            n_tsteps=100, 
            lr=1e-3, 
            weight_decay=1e-3,
            discriminator_weight=0.,
            discriminator_func_for_grad_weight=0.,
            id_dim=1,
            id_emb_dim=1,
            density_weight=0.,
            length_weight=1.,
        )

        # batch = next(iter(dataloader))
        # x0, x1, ids = batch
        try:
            data_gt = np.load(f"/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/dmae/data/neurips_results/toy/gt/{data_name}.npz", allow_pickle=True)
        except:
            print(f"CANNOT FIND FILE {data_name}.npz")
            missing.append(data_name)
            continue
        xbatch = torch.tensor(data_gt['start_points'], dtype=x.dtype, device=x.device)
        xendbatch = torch.tensor(data_gt['end_points'], dtype=x.dtype, device=x.device)
        x0 = xbatch
        x1 = xendbatch
        # xbatch = model.encoder.preprocessor.normalize(xbatch)
        # xendbatch = model.encoder.preprocessor.normalize(xendbatch)
        ids = torch.zeros((xbatch.size(0),1))
        # ids = torch.eye((xbatch.size(0)))

        # dataset = TensorDataset(xbatch, xendbatch, ids)
        # dataloader = DataLoader(dataset, batch_size=len(z), shuffle=True)

        def cc_func(x0, x1, t):
            return gbmodel.cc(x0, x1, t, ids)
        vectors = velocity(cc_func, gbmodel.ts, x0, x1)
        cc_pts = gbmodel.cc(x0, x1, gbmodel.ts, ids)
        vectors_flat = vectors.flatten(0,1)
        cc_pts_flat = cc_pts.flatten(0, 1)
        jac_flat = jacobian(gbmodel.func, cc_pts_flat)
        length_all = torch.sqrt((torch.einsum("nij,nj->ni", jac_flat, vectors_flat)**2).sum(axis=1))
        length_all = length_all.reshape(vectors.shape[0], vectors.shape[1])
        length = length_all.mean(axis=0)

        geods = (cc_pts_flat).reshape(cc_pts.shape)
        length2 = torch.sqrt(torch.diff(geods, axis=0)**2).sum(axis=-1).sum(axis=0)


        # plt.scatter(length2.detach().numpy(), data_gt['geodesic_lengths'])
        # plt.title(data_name)
        # plt.show()

        # gt_len = torch.tensor(data_gt['geodesic_lengths'])
        true_geod = torch.tensor(data_gt['geodesics'], dtype=torch.float32)
        gt_len = torch.sqrt(torch.diff(true_geod.permute(1,0,2), axis=0)**2).sum(axis=-1).sum(axis=0)
        corr = np.corrcoef(length.detach().numpy(), gt_len.cpu().numpy())[0,1]
        mse = ((length.detach().numpy() - gt_len.cpu().numpy())**2).mean()
        dist2geod = distance_to_geodesic_criterion_len(geods.permute(1,0,2), true_geod, lengths=gt_len).detach().numpy()

        res_list.append(dict(
            name=data_name,
            length_corr=corr,
            length_mse=mse,
            dist2geod=dist2geod
        ))
    except Exception as e:
        failed2.append(data_name)
        print(e)

res_df2 = pd.DataFrame(res_list)


  0%|          | 0/36 [00:00<?, ?it/s]/gpfs/gibbs/pi/krishnaswamy_smita/xingzhi/.conda_envs/geosink/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:269: UserWarning: Attribute 'preprocessor' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['preprocessor'])`.
  rank_zero_warn(
  3%|▎         | 1/36 [00:00<00:07,  4.76it/s]

Geod not found Series([], Name: run_id, dtype: object) torus_5_0.1
single positional indexer is out-of-bounds


 14%|█▍        | 5/36 [00:00<00:03, 10.28it/s]

Geod not found Series([], Name: run_id, dtype: object) torus_none_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) torus_15_0.7
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) torus_15_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) torus_50_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) torus_15_0.3
single positional indexer is out-of-bounds


 31%|███       | 11/36 [00:00<00:01, 16.82it/s]

...

Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_15_0.5
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.5
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_10_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_5_0.1
single positional indexer is out-of-bounds


 86%|████████▌ | 31/36 [00:02<00:00, 18.73it/s]

Geod not found Series([], Name: run_id, dtype: object) ellipsoid_15_0.7
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_none_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) hemisphere_15_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) hemisphere_none_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_50_0.1
single positional indexer is out-of-bounds


100%|██████████| 36/36 [00:02<00:00, 15.25it/s]

Geod not found Series([], Name: run_id, dtype: object) ellipsoid_15_0.1
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_15_0
single positional indexer is out-of-bounds
Geod not found Series([], Name: run_id, dtype: object) ellipsoid_15_0.3
single positional indexer is out-of-bounds


In [9]:
results_ours = pd.concat([res_df, res_df2], ignore_index=True)

In [10]:
results_ours.sort_values('name')

KeyError: 'name'

In [ ]:
results_ours

,name,length_corr,length_mse,dist2geod
0,torus_5_0.1,0.960356,9.875797,0.053779595
1,torus_none_0.1,0.987134,8.515339,0.059096955
2,torus_15_0.7,0.478949,14.321295,1.2238166
3,torus_50_0.1,0.933761,33.379780,0.105932415
4,torus_15_0.3,0.907351,18.993786,0.2710719
5,torus_15_0.5,0.821626,16.251606,0.590967
6,torus_10_0.1,0.931861,14.914821,0.039224662
7,saddle_15_0.1,0.877697,2.965390,0.07588364
8,saddle_15_0.7,0.084417,5.793678,2.3582518
9,saddle_15_0,0.962471,4.047969,0.019819994


In [ ]:
results_ours.to_csv('geodesics_density.csv', index=False)

In [ ]:
failed2

['torus_15_0.1', 'saddle_15_0.3', 'hemisphere_15_0.3']

: 